# Tarea 1: DS Lab
- Ricardo Arancibia
- José Díaz
- Javier Santibáñez

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## P1

In [107]:
df_all = pd.read_csv('data/raw/w{}/metrocuadrado_all_w{}.csv'.format(str(13),str(13)))
for NN in range(14,18):
    direc = 'data/raw/w{}/metrocuadrado_all_w{}.csv'.format(str(NN),str(NN))
    df_aux = pd.read_csv(direc)
    # df_all.merge(df_aux)
    df_all = pd.concat([df_all,df_aux], ignore_index= True)
df_all['Archivo'] = 'all' 
df_all.drop_duplicates(inplace=True, ignore_index= True)
print(df_all.shape)
df_all.head()

(16295, 9)


,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,all
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,all
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,all
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,all
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,all


In [108]:
df_fur = pd.read_csv('data/raw/w{}/metrocuadrado_furnished_w{}.csv'.format(str(13),str(13)))
for NN in range(14,18):
    direc = 'data/raw/w{}/metrocuadrado_furnished_w{}.csv'.format(str(NN),str(NN))
    df_aux = pd.read_csv(direc)
    # df_fur.merge(df_aux)
    df_fur = pd.concat([df_fur,df_aux], ignore_index= True)
df_fur['Archivo'] = 'furnished'
df_fur.drop_duplicates(inplace=True, ignore_index= True)
print(df_fur.shape)
df_fur.head()

(2099, 9)


,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo
0,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3.0,2.0,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,furnished
1,"Casa en Venta Y Arriendo, BELLAVISTA Bogotá D.C..",$1.300.000,3.0,2.0,72.0 m2,"hermosa casa en venta y en arriendo en, mandal...",https://www.metrocuadrado.com/inmueble/venta-c...,41.4,furnished
2,"Casa en Arriendo, PORTALES DEL NORTE Bogotá D.C..",$1.800.000,3.0,3.0,76.81 m2,Estrato 4. CASA TRIPLEX!! Exterior . Cuenta co...,https://www.metrocuadrado.com/inmueble/arriend...,41.3,furnished
3,"Casa en Venta Y Arriendo, SANTA SOFIA Bogotá D...",$1.500.000,5.0,3.0,76.22 m2,"Casa en Santa Sofia, excelente ubicación sobre...",https://www.metrocuadrado.com/inmueble/venta-c...,34.2,furnished
4,"Casa en Venta Y Arriendo, URBANIZACION SABANA ...",$145.000.000,3.0,2.0,77.0 m2,"Casa esquinera, de tres niveles, se venden mue...",https://www.metrocuadrado.com/inmueble/venta-c...,62.5,furnished


In [311]:
df = pd.concat([df_all,df_fur],ignore_index= True)
print(df.shape)
df.head()

(18394, 9)


,property_type|rent_type|location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo
0,"Casa en Arriendo, LA SOLEDAD NORTE Bogotá D.C..",$1.050.000,3,2,63.0 m2,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,all
1,"Casa en Arriendo, CIUDAD SALITRE SUR-ORIENTAL ...",$3.930.000,3,2,100.0 m2,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,all
2,"Casa en Arriendo, VILLA DE LOS ALPES II SECTOR...",$750.000,3,1,90.0 m2,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,all
3,"Casa en Arriendo, PARIS GAITAN Bogotá D.C..",$3.400.000,5,4,71.0 m2,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,all
4,"Casa en Arriendo, BALCONES DE ORIENTE Bogotá D...",$1.560.000,4,3,96.0 m2,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,all


In [312]:
df.dropna(subset=['price','n_rooms','n_bath','surface'], inplace=True)
df.reset_index(drop=True,inplace=True)
print(df.shape)

(18143, 9)


In [313]:
df['price'] = [x.replace('$','') for x in df['price']]
df['price'] = [x.replace('.','') for x in df['price']]
df['surface'] = [x.replace(' m2','') for x in df['surface']]
df['price'] = df['price'].astype('float64')
df['surface'] = df['surface'].astype('float64')
for i in range(len(df)):
    x = df['n_bath'][i]
    y = df['n_rooms'][i]
    if type(x)== float:
        df['n_bath'][i] = str(int(x)) # 3.0 <-> '3'
    if type(y)== float:
        df['n_rooms'][i] = str(int(y))
# df.head()

In [314]:
df_aux = df['property_type|rent_type|location'].str.split(pat=',',expand =True)
df_aux[0] = [x.replace(' ',', ', 1) for x in df_aux[0]]
df_aux = pd.concat([df_aux[0].str.split(pat=',',expand =True), df_aux[1]], axis=1, ignore_index=True)
df_aux.columns = ['property_type','rent_type','location']
df_aux['rent_type'] = [x.replace('en ','') for x in df_aux['rent_type']]
# df_aux
df = pd.concat([df_aux,df.drop(columns=['property_type|rent_type|location'])], axis=1)
df.head()

,property_type,rent_type,location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo
0,Casa,Arriendo,LA SOLEDAD NORTE Bogotá D.C..,1050000.0,3,2,63.0,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,all
1,Casa,Arriendo,CIUDAD SALITRE SUR-ORIENTAL Bogotá D.C..,3930000.0,3,2,100.0,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,all
2,Casa,Arriendo,VILLA DE LOS ALPES II SECTOR Bogotá D.C..,750000.0,3,1,90.0,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,all
3,Casa,Arriendo,PARIS GAITAN Bogotá D.C..,3400000.0,5,4,71.0,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,all
4,Casa,Arriendo,BALCONES DE ORIENTE Bogotá D.C..,1560000.0,4,3,96.0,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,all


In [315]:
df['price by m2'] = df['price']/df['surface']

In [316]:
lista = []
for x in df['url']:
    if x.find('-garajes')==-1:
        lista.append('0')
    else:
        lista.append(x[x.find('banos-')+6: x.find('-garajes')])
df['n_garajes'] = lista

In [317]:
df['tipo_producto'] = np.zeros(len(df)) # categoria 0
indices=df.query('80<=surface <120 and property_type=="Casa"').index
df.loc[indices,'tipo_producto'] = 1
indices=df.query('120<=surface <180 and property_type=="Casa"').index
df.loc[indices,'tipo_producto'] = 2
indices=df.query('180<=surface <240 and property_type=="Casa"').index
df.loc[indices,'tipo_producto'] = 3
indices=df.query('240<=surface <360 and property_type=="Casa"').index
df.loc[indices,'tipo_producto'] = 4
indices=df.query('360<=surface <=460 and property_type=="Casa"').index
df.loc[indices,'tipo_producto'] = 5
indices=df.query('40<=surface <60 and property_type=="Apartamento"').index
df.loc[indices,'tipo_producto'] = 6
indices=df.query('60<=surface <80 and property_type=="Apartamento"').index
df.loc[indices,'tipo_producto'] = 7
indices=df.query('80<=surface <=120 and property_type=="Apartamento"').index
df.loc[indices,'tipo_producto'] = 8

In [318]:
# 5
df_upz = pd.read_csv('data/asignacion_upz/barrio-upz.csv')
df_upz.rename(columns={'pro_location':'location'}, inplace=True)
# print(df_upz.shape)

In [319]:
df['location'] = df['location'].apply(str.lower)
df['location'] = [x.replace(' bogotá d.c..','') for x in df['location']]
df['location'] = df['location'].apply(str.strip)

df = pd.merge(df, df_upz, how='left', on='location')
df.head()

,property_type,rent_type,location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,Archivo,price by m2,n_garajes,tipo_producto,UPlCodigo,UPlTipo,UPlNombre,UPlArea
0,Casa,Arriendo,la soledad norte,1050000.0,3,2,63.0,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,all,16666.666667,1,0.0,NaN,NaN,NaN,NaN
1,Casa,Arriendo,ciudad salitre sur-oriental,3930000.0,3,2,100.0,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,all,39300.000000,1,1.0,NaN,NaN,NaN,NaN
2,Casa,Arriendo,villa de los alpes ii sector,750000.0,3,1,90.0,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,all,8333.333333,0,1.0,NaN,NaN,NaN,NaN
3,Casa,Arriendo,paris gaitan,3400000.0,5,4,71.0,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,all,47887.323944,0,0.0,UPZ29,1.0,MINUTO DE DIOS,3.732645e+06
4,Casa,Arriendo,balcones de oriente,1560000.0,4,3,96.0,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,all,16250.000000,1,1.0,UPZ12,1.0,TOBERIN,2.906631e+06


In [330]:
df.rename(columns={'UPlCodigo':'UPZ_Codigo'}, inplace=True)
print('Numero de observaciones a los que no se les puede adjuntar un codigo UPZ: ', df['UPZ_Codigo'].isna().sum())
print("Porcentaje asignable: ", round((len(df)-df['UPZ_Codigo'].isna().sum())/len(df)*100,1))
df.dropna(subset=['UPZ_Codigo'], inplace=True)
df.reset_index(drop=True,inplace=True)

Numero de observaciones a los que no se les puede adjuntar un codigo UPZ:  2031
Porcentaje asignable:  88.8


In [331]:
df_est_pobl = pd.read_csv('data/estadisticas_upz/estadisticas_poblacion.csv',index_col=0)
df_est_pobl.rename(columns={'upz':'UPZ_Codigo'}, inplace=True)
df = pd.merge(df, df_est_pobl, how='left', on='UPZ_Codigo')
df.head()

,property_type,rent_type,location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,...,UPlNombre,UPlArea,nomupz,personas,trabajoinf_ninos_5_17_anos_perc,trabajoinfampliado_ninos_5_17_anos_perc,jovenes_14_24_anos_nini_perc,indice_envegecimiento,jefe_mujer_perc,adultos_mayores_pobres_perc
0,Casa,Arriendo,la soledad norte,1050000.0,3,2,63.0,Excelente casa cerca de la calle 80 y Avenida ...,https://www.metrocuadrado.com/inmueble/arriend...,25.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Casa,Arriendo,ciudad salitre sur-oriental,3930000.0,3,2,100.0,Se arrienda casa bonita totalmente amoblada co...,https://www.metrocuadrado.com/inmueble/arriend...,57.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Casa,Arriendo,villa de los alpes ii sector,750000.0,3,1,90.0,CASA DE TRES NIVELES TRES ALCOBAS SALA COMEDOR...,https://www.metrocuadrado.com/inmueble/arriend...,12.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Casa,Arriendo,paris gaitan,3400000.0,5,4,71.0,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,...,MINUTO DE DIOS,3.732645e+06,MINUTO DE DIOS,156375.0,1.1,0.8,12.6,71.1,37.073314,8.406360
4,Casa,Arriendo,balcones de oriente,1560000.0,4,3,96.0,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,...,TOBERIN,2.906631e+06,TOBERIN,50037.0,1.8,16.7,9.1,54.7,36.279630,5.420442


In [334]:
df_est_ind = pd.read_csv('data/estadisticas_upz/indice_inseguridad.csv',index_col=0)
df_est_ind.rename(columns={'UPlCodigo':'UPZ_Codigo'}, inplace=True)
df = pd.merge(df, df_est_ind, how='left', on='UPZ_Codigo')
df.head()

,property_type,rent_type,location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,...,nomupz,personas,trabajoinf_ninos_5_17_anos_perc,trabajoinfampliado_ninos_5_17_anos_perc,jovenes_14_24_anos_nini_perc,indice_envegecimiento,jefe_mujer_perc,adultos_mayores_pobres_perc,UPlNombre2,indice_inseguridad
0,Casa,Arriendo,paris gaitan,3400000.0,5,4,71.0,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,...,MINUTO DE DIOS,156375.0,1.1,0.8,12.6,71.1,37.073314,8.406360,MINUTO DE DIOS,1.029160
1,Casa,Arriendo,balcones de oriente,1560000.0,4,3,96.0,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,...,TOBERIN,50037.0,1.8,16.7,9.1,54.7,36.279630,5.420442,TOBERIN,3.166227
2,Casa,Arriendo,tibabuyes,1100000.0,3,2,69.0,"Casa en conjunto cerrado, recién remodelada, c...",https://www.metrocuadrado.com/inmueble/arriend...,32.4,...,SUBA,158990.0,0.4,1.8,17.2,24.8,34.847797,13.750649,SUBA,3.225806
3,Casa,Arriendo,paris gaitan,3400000.0,4,4,71.0,casa ezquinera tres niveles excelente ubicacio...,https://www.metrocuadrado.com/inmueble/arriend...,60.9,...,MINUTO DE DIOS,156375.0,1.1,0.8,12.6,71.1,37.073314,8.406360,MINUTO DE DIOS,1.029160
4,Casa,Arriendo,campo alegre,578000.0,3,2,62.0,Casa de cuatro niveles amplias habitaciones zo...,https://www.metrocuadrado.com/inmueble/arriend...,40.5,...,SANTA BARBARA,115124.0,1.5,2.0,5.2,127.3,38.443910,3.840225,SANTA BARBARA,1.161103


In [341]:
df_est_por = pd.read_csv('data/estadisticas_upz/porcentaje_areas_verdes.csv',index_col=0)
df_est_por.rename(columns={'cod_upz':'UPZ_Codigo'}, inplace=True)
df_est_por['UPZ_Codigo'] = df_est_por['UPZ_Codigo'].astype(int)
df_est_por['UPZ_Codigo'] = ['UPZ'+str(x) for x in df_est_por['UPZ_Codigo']]
df = pd.merge(df, df_est_por, how='left', on='UPZ_Codigo')
df.head()

,property_type,rent_type,location,price,n_rooms,n_bath,surface,details,url,metrocuadrado_index,...,trabajoinf_ninos_5_17_anos_perc,trabajoinfampliado_ninos_5_17_anos_perc,jovenes_14_24_anos_nini_perc,indice_envegecimiento,jefe_mujer_perc,adultos_mayores_pobres_perc,UPlNombre2,indice_inseguridad,upz,areas_verdes_perc
0,Casa,Arriendo,paris gaitan,3400000.0,5,4,71.0,casa ezquinera excelente ubicacion cinco alcob...,https://www.metrocuadrado.com/inmueble/arriend...,56.0,...,1.1,0.8,12.6,71.1,37.073314,8.406360,MINUTO DE DIOS,1.029160,Minuto de Dios,37.852289
1,Casa,Arriendo,balcones de oriente,1560000.0,4,3,96.0,"Excelente casa de tres niveles, buena ubicació...",https://www.metrocuadrado.com/inmueble/arriend...,47.1,...,1.8,16.7,9.1,54.7,36.279630,5.420442,TOBERIN,3.166227,Toberin,32.395663
2,Casa,Arriendo,tibabuyes,1100000.0,3,2,69.0,"Casa en conjunto cerrado, recién remodelada, c...",https://www.metrocuadrado.com/inmueble/arriend...,32.4,...,0.4,1.8,17.2,24.8,34.847797,13.750649,SUBA,3.225806,Suba,45.096719
3,Casa,Arriendo,paris gaitan,3400000.0,4,4,71.0,casa ezquinera tres niveles excelente ubicacio...,https://www.metrocuadrado.com/inmueble/arriend...,60.9,...,1.1,0.8,12.6,71.1,37.073314,8.406360,MINUTO DE DIOS,1.029160,Minuto de Dios,37.852289
4,Casa,Arriendo,campo alegre,578000.0,3,2,62.0,Casa de cuatro niveles amplias habitaciones zo...,https://www.metrocuadrado.com/inmueble/arriend...,40.5,...,1.5,2.0,5.2,127.3,38.443910,3.840225,SANTA BARBARA,1.161103,Santa Barbara,25.466350


In [347]:
df['densidad_pobl'] = df['personas']/df['surface']